In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.models as models

import PIL
from PIL import Image
import requests
from io import BytesIO

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
def load_image_from_url(url, max_size=512):
    """Load image from URL and preprocess it"""
    response = requests.get(url)
    image = Image.open(BytesIO(response.content)).convert('RGB')

    # Resize image while maintaining aspect ratio
    if max(image.size) > max_size:
        image.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)

    return image

In [ ]:
def image_to_tensor(image, device):
    """Convert PIL image to tensor"""
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Add batch dimension
    tensor = transform(image).unsqueeze(0)
    return tensor.to(device)

In [ ]:
def tensor_to_image(tensor):
    """Convert tensor back to PIL image for visualization"""
    # Remove batch dimension
    tensor = tensor.squeeze(0).cpu()

    # Denormalize
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    tensor = tensor * std + mean

    # Clamp values to [0, 1]
    tensor = torch.clamp(tensor, 0, 1)

    # Convert to PIL image
    transform = transforms.ToPILImage()
    image = transform(tensor)
    return image

In [ ]:
class VGGFeatureExtractor(nn.Module):
    """VGG19 feature extractor for style transfer"""

    def __init__(self):
        super(VGGFeatureExtractor, self).__init__()

        # Load pre-trained VGG19
        vgg = models.vgg19(pretrained=True).features

        # We only need the convolutional layers
        self.features = vgg

        # Freeze the parameters
        for param in self.parameters():
            param.requires_grad = False

        # Define which layers to use for content and style
        # Content: conv4_2 (relu4_2)
        # Style: conv1_1, conv2_1, conv3_1, conv4_1, conv5_1
        self.content_layers = ['21']  # conv4_2
        self.style_layers = ['0', '5', '10', '19', '28']  # conv1_1, conv2_1, conv3_1, conv4_1, conv5_1

    def forward(self, x):
        """Extract features from specified layers"""
        content_features = {}
        style_features = {}

        for name, layer in self.features._modules.items():
            x = layer(x)

            if name in self.content_layers:
                content_features[name] = x

            if name in self.style_layers:
                style_features[name] = x

        return content_features, style_features


In [ ]:
def content_loss(target_features, content_features):
    """Calculate content loss"""
    loss = 0
    for layer in content_features:
        loss += torch.mean((target_features[layer] - content_features[layer]) ** 2)
    return loss

In [ ]:
def gram_matrix(tensor):
    """Calculate Gram matrix for style loss"""
    batch_size, channels, height, width = tensor.size()

    # Reshape tensor to (batch_size, channels, height*width)
    tensor = tensor.view(batch_size, channels, height * width)

    # Calculate Gram matrix
    gram = torch.bmm(tensor, tensor.transpose(1, 2))

    # Normalize by number of elements
    gram = gram / (channels * height * width)

    return gram


In [ ]:
def style_loss(target_features, style_features):
    """Calculate style loss using Gram matrices"""
    loss = 0
    for layer in style_features:
        target_gram = gram_matrix(target_features[layer])
        style_gram = gram_matrix(style_features[layer])
        loss += torch.mean((target_gram - style_gram) ** 2)
    return loss

In [ ]:
def main():
    # Get image URLs from user
    print("Neural Style Transfer")
    print("=" * 50)

    content_url = input("Enter the URL of your content image: ").strip()
    style_url = input("Enter the URL of your style image: ").strip()

    if not content_url or not style_url:
        print("Error: Both content and style image URLs are required!")
        return

    # Get parameters from user (with defaults)
    print("\nOptional parameters (press Enter for defaults):")

    content_weight_input = input(f"Content weight (default: 1e4): ").strip()
    content_weight = float(content_weight_input) if content_weight_input else 1e4

    style_weight_input = input(f"Style weight (default: 1e6): ").strip()
    style_weight = float(style_weight_input) if style_weight_input else 1e6

    learning_rate_input = input(f"Learning rate (default: 0.01): ").strip()
    learning_rate = float(learning_rate_input) if learning_rate_input else 0.01

    num_iterations_input = input(f"Number of iterations (default: 500): ").strip()
    num_iterations = int(num_iterations_input) if num_iterations_input else 500

    show_every_input = input(f"Show progress every N iterations (default: 50): ").strip()
    show_every = int(show_every_input) if show_every_input else 50

    print(f"\nUsing parameters:")
    print(f"Content weight: {content_weight}")
    print(f"Style weight: {style_weight}")
    print(f"Learning rate: {learning_rate}")
    print(f"Iterations: {num_iterations}")
    print(f"Show progress every: {show_every} iterations")

    # Load images
    print("\nLoading images...")
    try:
        content_image = load_image_from_url(content_url)
        print("✓ Content image loaded successfully")
    except Exception as e:
        print(f"Error loading content image: {e}")
        return

    try:
        style_image = load_image_from_url(style_url)
        print("✓ Style image loaded successfully")
    except Exception as e:
        print(f"Error loading style image: {e}")
        return

    # Convert images to tensors
    content_tensor = image_to_tensor(content_image, device)
    style_tensor = image_to_tensor(style_image, device)

    # Resize style image to match content image size
    style_tensor = nn.functional.interpolate(style_tensor, size=content_tensor.shape[2:], mode='bilinear', align_corners=False)

    # Initialize the target image (start with content image)
    target_tensor = content_tensor.clone().requires_grad_(True)

    # Initialize the feature extractor
    print("Loading VGG19 model...")
    vgg = VGGFeatureExtractor().to(device)
    vgg.eval()

    # Extract features from content and style images
    print("Extracting features...")
    with torch.no_grad():
        content_features, _ = vgg(content_tensor)
        _, style_features = vgg(style_tensor)

    # Setup optimizer
    optimizer = optim.Adam([target_tensor], lr=learning_rate)

    print("\nStarting style transfer...")
    print("-" * 50)

    # Style transfer optimization loop
    for iteration in range(num_iterations):
        # Forward pass
        target_content_features, target_style_features = vgg(target_tensor)

        # Calculate losses
        c_loss = content_loss(target_content_features, content_features)
        s_loss = style_loss(target_style_features, style_features)
        total_loss = content_weight * c_loss + style_weight * s_loss

        # Backward pass
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        # Print progress
        if iteration % show_every == 0:
            print(f"Iteration {iteration:4d} | Total Loss: {total_loss.item():.2f} | "
                  f"Content Loss: {c_loss.item():.2f} | Style Loss: {s_loss.item():.2f}")

    print("\nStyle transfer completed!")

    # Convert final result to image
    print("Displaying final result...")
    final_image = tensor_to_image(target_tensor)

    import matplotlib.pyplot as plt

    print("✓ Final stylized image displayed!")
    plt.imshow(final_image)
    plt.axis('off')
    plt.title("Final Stylized Image")
    plt.show()



In [ ]:
if __name__ == "__main__":
    main()

Neural Style Transfer
Enter the URL of your content image: https://images.unsplash.com/photo-1544005313-94ddf0286df2?ixlib=rb-4.0.3&auto=format&fit=crop&w=800&q=60
Enter the URL of your style image: https://upload.wikimedia.org/wikipedia/commons/f/f4/The_Scream.jpg

Optional parameters (press Enter for defaults):
Content weight (default: 1e4): 
Style weight (default: 1e6): 
Learning rate (default: 0.01): 
Number of iterations (default: 500): 
Show progress every N iterations (default: 50): 

Using parameters:
Content weight: 10000.0
Style weight: 1000000.0
Learning rate: 0.01
Iterations: 500
Show progress every: 50 iterations

Loading images...
✓ Content image loaded successfully
Error loading style image: cannot identify image file <_io.BytesIO object at 0x794551d330b0>
